In [1]:
import torch
#import clip
#from PIL import Image
import zipfile
from pathlib import Path
import json
import math
from collections import defaultdict
from torch.nn.functional import pad
from transformers import AutoTokenizer, AutoModelForMaskedLM
from tqdm.notebook import tqdm
import pickle

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#model, preprocess = clip.load("ViT-B/32", device=device)

#clip_im = preprocess(images[0]).unsqueeze(0).to(device)

#with torch.no_grad():
#    image_features = model.encode_image(clip_im)
    
#image_features

In [3]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('AI-Growth-Lab/PatentSBERTa', device=device)
embeddings = model.encode(sentences)
print(embeddings)

[[ 0.03105076 -0.41703615 -0.38322818 ... -0.21281055 -0.00351022
  -0.27905735]
 [ 0.11301837 -0.19698262 -0.25981197 ... -0.16589701 -0.30437657
  -0.2781442 ]]


In [4]:
def tokenize_string(string, model):
    print(len(string))
    tokenized = model.tokenize([string])
    input_ids = tokenized['input_ids']
    if len(input_ids[0]) >= 512:
        words = string.split()
        n_words = len(words)
        first_half = words[:n_words//2]
        second_half = words[n_words//2:]
        first_tokenized = tokenize_string(' '.join(first_half), model)
        second_tokenized = tokenize_string(' '.join(second_half), model)
        concatentated_input_ids = torch.cat([first_tokenized['input_ids'], second_tokenized['input_ids']], dim=-1)
        concatentated_attention = torch.cat([first_tokenized['attention_mask'], second_tokenized['attention_mask']], dim=-1)
        tokenized = {'input_ids': concatentated_input_ids, 'attention_mask': concatentated_attention}
    return tokenized

    

In [5]:
tokenize_string(' '.join(['foo']*100000), model)['input_ids'].shape

399999
199999
99999
49999
24999
12499
6247
3123
1559
1563
3123
1559
1563
6251
3123
1559
1563
3127
1563
1563
12499
6247
3123
1559
1563
3123
1559
1563
6251
3123
1559
1563
3127
1563
1563
24999
12499
6247
3123
1559
1563
3123
1559
1563
6251
3123
1559
1563
3127
1563
1563
12499
6247
3123
1559
1563
3123
1559
1563
6251
3123
1559
1563
3127
1563
1563
49999
24999
12499
6247
3123
1559
1563
3123
1559
1563
6251
3123
1559
1563
3127
1563
1563
12499
6247
3123
1559
1563
3123
1559
1563
6251
3123
1559
1563
3127
1563
1563
24999
12499
6247
3123
1559
1563
3123
1559
1563
6251
3123
1559
1563
3127
1563
1563
12499
6247
3123
1559
1563
3123
1559
1563
6251
3123
1559
1563
3127
1563
1563
99999
49999
24999
12499
6247
3123
1559
1563
3123
1559
1563
6251
3123
1559
1563
3127
1563
1563
12499
6247
3123
1559
1563
3123
1559
1563
6251
3123
1559
1563
3127
1563
1563
24999
12499
6247
3123
1559
1563
3123
1559
1563
6251
3123
1559
1563
3127
1563
1563
12499
6247
3123
1559
1563
3123
1559
1563
6251
3123
1559
1563
3127
1563
1563
49999
24

torch.Size([1, 100512])

In [6]:
def encode_text(text, model, device=device, batch_size=16, window_length=512):
    step_size = int(window_length/2)
    with torch.no_grad():
        tokenization_results = tokenize_string(text, model)
        tokenized_text = tokenization_results['input_ids'][0]
        attention_mask = tokenization_results['attention_mask'][0]
        n_tokens = len(tokenized_text)
        if n_tokens < window_length:
            tokenized_text = pad(tokenized_text, (0, window_length-n_tokens))
            attention_mask = pad(attention_mask, (0, window_length-n_tokens))
            n_tokens = len(tokenized_text)
        padded_length = math.ceil((len(tokenized_text) - window_length)/(step_size))*step_size + window_length
        tokenized_text = pad(tokenized_text, (0, padded_length-n_tokens))
        attention_mask = pad(attention_mask, (0, padded_length-n_tokens))
        unfolded_text = tokenized_text.unfold(0, window_length, step_size)
        unfolded_mask = attention_mask.unfold(0, window_length, step_size)

        num_batches = math.ceil(len(unfolded_text)/batch_size)
        encoded_texts = []
        for i in range(num_batches):
            batch_text = unfolded_text[i*batch_size:(i+1)*batch_size]
            batch_mask = unfolded_mask[i*batch_size:(i+1)*batch_size]
            batch = {'input_ids': batch_text.to(device=device), 'attention_mask': batch_mask.to(device=device)}
            out_features = model.forward(batch) # Weird that the SentenceTransformer explicitly calls forward
            embeddings = out_features['sentence_embedding']
            encoded_text = embeddings.detach().cpu()
            encoded_texts.append(encoded_text)
        encoded_texts = torch.cat(encoded_texts, dim=0)
        summary_encoding = torch.sum(encoded_text, dim=0)
        return summary_encoding.detach().cpu().numpy()
    

In [85]:
tokenized = model.tokenize(['foo is the bar'])
tokenized = {k: v.to(device) for k,v in tokenized.items()}

In [8]:
encode_text('foo is the greatest', model)

array([-1.53004497e-01, -1.37338325e-01, -3.48920733e-01, -6.89426661e-02,
        1.37642637e-01, -5.32362871e-02,  2.41966685e-03, -2.41582952e-02,
        1.13605186e-02,  2.35071123e-01,  1.32103533e-01, -1.13797761e-01,
       -1.05135128e-01,  8.74493271e-02,  2.18807504e-01,  1.88617453e-01,
        2.22857833e-01,  2.00768590e-01,  4.91114855e-02, -2.55568027e-01,
       -2.10523114e-01, -1.73775867e-01, -1.53341979e-01, -6.17561601e-02,
       -1.57454148e-01,  2.49930769e-01,  2.81612307e-01,  3.32992822e-02,
       -5.40803552e-01, -1.02503709e-01, -1.07836559e-01,  1.23734720e-01,
       -1.21516764e-01,  1.94661453e-01, -1.01327809e-04, -3.35867822e-01,
        2.35678300e-01,  8.34209323e-02,  2.12049738e-01, -3.28996480e-01,
       -4.82365370e-01,  3.06632072e-01, -3.97826582e-01,  1.44867346e-01,
       -1.18524373e-01, -3.01666081e-01, -2.65770499e-02, -3.15617353e-01,
       -1.48784369e-02,  1.55304119e-01,  2.77757406e-01, -4.22556818e-01,
       -3.29479426e-01, -

In [7]:
def encode_packaged_patents(packaged, doc_start=None, doc_end=None, batch_size=16, window_length=512):
    with zipfile.ZipFile(packaged) as z:
        document_files = defaultdict(lambda: {'patent_info': None, 'images': []})
        n_infos = 0
        for info in z.infolist():
            n_infos += 1
            patent, filename = info.filename.split('/')
            base, ext = filename.split('.')
            if ext == 'json':
                document_files[patent]['patent_info'] = info
            elif ext == 'tif':
                document_files[patent]['images'].append(info)
        
        
        text_patents = []
        
        for patent, infos in tqdm(sorted(document_files.items())[737:740]):
            with z.open(infos['patent_info']) as fp:
                patent_info = json.load(fp)
                

            abstract = patent_info['abstract']['en']
            description = patent_info['description']['en']
            claims = patent_info['claims']['en']
            applicants = patent_info['applicants']
            publication_date = patent_info['publication_date']
            ipc_classes = patent_info['ipc_classes']
            try:
                vectors = dict(abstract_vector=encode_text(abstract, model, batch_size=batch_size, window_length=window_length),
                               description_vector=encode_text(description, model, batch_size=batch_size, window_length=window_length),
                               claims_vector = encode_text(claims, model, batch_size=batch_size, window_length=window_length))
            except RuntimeError:
                continue    
            patent_repr = dict(patent_number=patent,
            vectors=vectors,
                            applicants=','.join(applicants),
                            publication_date=publication_date, 
                            ipc_classes=ipc_classes)
                    
            
            text_patents.append(patent_repr)

            
    return text_patents

        

In [8]:
packaged = Path('F:/datasets/PRV_KEMI_DATA/packaged_patents/english_random_sample.zip')
text_patents = encode_packaged_patents(packaged)
output_dir = Path('F:/datasets/PRV_KEMI_DATA/strans_repr/')
output_dir.mkdir(exist_ok=True)
with open(output_dir / 'patentsbert_no_images_random.pkl', 'wb') as fp:
    pickle.dump(text_patents, fp)

  0%|          | 0/3 [00:00<?, ?it/s]

517
98289
43133
23002
11637
6730
3284
1602
1681
841
839
3445
1732
825
906
1712
833
878
4906
2390
1185
1204
2515
1311
1203
11364
5562
2659
1294
1364
2902
1306
1595
5801
2854
1322
1531
700
830
2946
1605
1340
20130
9232
4751
2441
1292
1148
2309
1319
989
4480
2224
962
1261
2255
1107
1147
10897
5383
2461
1256
1204
2921
1489
1431
5513
2627
1338
1288
2885
1448
1436
55149
24707
12971
6502
3443
1255
2187
652
1534
1280
1127
1032
26
1005
12
992
980
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0
973
0


In [ ]:
len(text_patents)

In [8]:
collated_representations = defaultdict(list)
for document, vectors in sorted(image_patents.items()):
    collated_representations['patent_numbers'].append(document)
    for name, value in vectors.items():
        collated_representations[name].append(value)




In [9]:
collated_representations['patent_numbers']

['EP2336221.A1',
 'EP2341394.A1',
 'EP2341396.A1',
 'EP2343336.A1',
 'EP2345541.A2',
 'EP2345684.A1',
 'EP2348079.A1',
 'EP2354181.A1',
 'EP2354185.A1',
 'EP2354199.A1',
 'EP2354855.A1',
 'EP2357014.A1',
 'EP2357162.A1',
 'EP2357530.A2',
 'EP2359702.A1',
 'EP2359820.A1',
 'EP2361941.A1',
 'EP2361942.A1',
 'EP2362266.A2',
 'EP2363431.A1',
 'EP2363434.A1',
 'EP2364844.A2',
 'EP2364859.A2',
 'EP2365012.A1',
 'EP2365015.A1',
 'EP2365016.A1',
 'EP2365029.A2',
 'EP2365036.A1',
 'EP2365039.A1',
 'EP2365389.A1',
 'EP2366546.A2',
 'EP2366554.A2',
 'EP2366723.A2',
 'EP2366732.A2',
 'EP2366744.A1',
 'EP2368430.A2',
 'EP2368476.A2',
 'EP2368915.A1',
 'EP2368916.A1',
 'EP2368930.A1',
 'EP2371221.A2',
 'EP2371910.A1',
 'EP2371911.A1',
 'EP2371935.A1',
 'EP2372442.A2',
 'EP2374831.A1',
 'EP2374838.A1',
 'EP2374853.A1',
 'EP2374917.A1',
 'EP2375279.A2',
 'EP2377509.A1',
 'EP2377511.A1',
 'EP2377513.A1',
 'EP2377612.A1',
 'EP2377899.A1',
 'EP2378364.A1',
 'EP2380555.A1',
 'EP2382962.A1',
 'EP2383053.A1

In [32]:
import numpy as np
numpy_collated = {name: np.array(v) for name,v in collated_representations.items()}

C:\Users\Erik\AppData\Local\Temp/ipykernel_47820/2035335349.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  numpy_collated = {name: np.array(v) for name,v in collated_representations.items()}


In [204]:
p = Path('F:/datasets/PRV_KEMI_DATA/patents/netto_list/EP2330461NWA1.zip')

names = []
with zipfile.ZipFile(p) as z_f:
    print()
    
    
    tifs = [name for name in z_f.namelist() if name.split('.')[-1] == 'tif']
    for tif in tifs:
        with torch.no_grad():
            images = []
            with z_f.open(tif) as im_fp:
                names.append(tif)
                image = Image.open(im_fp)
                clip_im = preprocess(image).to(device)
                images.append(clip_im)
            images = torch.stack(images)
            encoded_images = model.encode_image(images)
            summary_image_vec = encoded_images.sum(dim=0)
    json_text_fn = 'EP3399368.A1/patent_info.json'
    with z.open('EP3399368.A1/patent_info.json') as fp:
        text = json.load(fp)
    #document_files = dict()
    #for info in z.infolist():
    #    patent, *_ = info.filename.split('/')
    



In [213]:
document_files

defaultdict(<function __main__.<lambda>()>,
            {'EP3399368.A1': {'patent_info': <ZipInfo filename='EP3399368.A1/patent_info.json' filemode='?rw-------' file_size=177939>,
              'images': [<ZipInfo filename='EP3399368.A1/imgaf001.tif' filemode='?rw-------' file_size=11022>,
               <ZipInfo filename='EP3399368.A1/imgb0001.tif' filemode='?rw-------' file_size=1334>,
               <ZipInfo filename='EP3399368.A1/imgb0002.tif' filemode='?rw-------' file_size=1887>,
               <ZipInfo filename='EP3399368.A1/imgb0003.tif' filemode='?rw-------' file_size=753>,
               <ZipInfo filename='EP3399368.A1/imgb0004.tif' filemode='?rw-------' file_size=5305>,
               <ZipInfo filename='EP3399368.A1/imgb0005.tif' filemode='?rw-------' file_size=4406>,
               <ZipInfo filename='EP3399368.A1/imgb0006.tif' filemode='?rw-------' file_size=4492>,
               <ZipInfo filename='EP3399368.A1/imgb0007.tif' filemode='?rw-------' file_size=5384>,
          

In [39]:
torch.linalg.norm(encoded_images, dim=-1)

tensor([ 8.8828,  9.0703,  8.8672,  9.3750,  9.0469,  9.6797, 10.0000, 10.2266,
         9.4922], device='cuda:0', dtype=torch.float16,
       grad_fn=<CopyBackwards>)

In [40]:
normed_images = encoded_images/torch.linalg.norm(encoded_images, dim=-1, keepdim=True)

In [71]:
normed_images.shape

torch.Size([9, 512])

In [42]:
normed_images @ normed_images.T

tensor([[0.9990, 0.9639, 0.9404, 0.7676, 0.5195, 0.4124, 0.4485, 0.4331, 0.4888],
        [0.9639, 0.9990, 0.9634, 0.8169, 0.5474, 0.4343, 0.4651, 0.4578, 0.5020],
        [0.9404, 0.9634, 0.9990, 0.8047, 0.5205, 0.4126, 0.4490, 0.4387, 0.4817],
        [0.7676, 0.8169, 0.8047, 1.0000, 0.6279, 0.5361, 0.5640, 0.5684, 0.5479],
        [0.5195, 0.5474, 0.5205, 0.6279, 1.0010, 0.8076, 0.8516, 0.8247, 0.8687],
        [0.4124, 0.4343, 0.4126, 0.5361, 0.8076, 1.0000, 0.9307, 0.9390, 0.7363],
        [0.4485, 0.4651, 0.4490, 0.5640, 0.8516, 0.9307, 0.9995, 0.9487, 0.7773],
        [0.4331, 0.4578, 0.4387, 0.5684, 0.8247, 0.9390, 0.9487, 1.0000, 0.7495],
        [0.4888, 0.5020, 0.4817, 0.5479, 0.8687, 0.7363, 0.7773, 0.7495, 1.0010]],
       device='cuda:0', dtype=torch.float16, grad_fn=<MmBackward0>)

In [43]:
names

['imgaf001.tif',
 'imgf0001.tif',
 'imgf0002.tif',
 'imgf0003.tif',
 'srep0001.tif',
 'srep0002.tif',
 'srep0003.tif',
 'srep0004.tif',
 'srep0005.tif']

In [65]:
description = text['description']['en']
abstract = text['abstract']['en']
claims = text['claims']['en']

tensor([[49406,  7582,  1570,   314,   271,   271,   271,   272,   316,   518,
          2881, 23607, 36064,   531,   320, 22337,   671,  1448,   533,  2026,
           531,  7529, 21264,  2046,   601, 27919,  3152, 21264,  1977, 27919,
          3683, 10922,   537,   968,  3313, 21264,  1977,  1153, 13207,  3816,
         10922,   537,   550, 16822,  3550, 29445,  1197,   518,  2022,   530,
         49407,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]])

In [123]:
description_words = description.split()

['TECHNICAL',
 'FIELD',
 '[0001]',
 'The',
 'present',
 'invention',
 'relates',
 'to',
 'a',
 'laminate',
 'which',
 'is',
 'used',
 'to',
 'produce',
 'lenses',
 'such',
 'as',
 'polarizing',
 'lenses',
 'having',
 'polarization',
 'properties',
 'and',
 'sunglass',
 'lenses',
 'having',
 'photochromic',
 'properties',
 'and',
 'an',
 'optical',
 'article',
 'comprising',
 'the',
 'laminate.',
 'BACKGROUND',
 'ART',
 '[0002]',
 'Demand',
 'for',
 'plastic',
 'substrates',
 'composed',
 'of',
 'polycarbonate',
 'having',
 'transparency',
 'and',
 'excellent',
 'impact',
 'resistance',
 'for',
 'use',
 'in',
 'sunglasses',
 'having',
 'glare',
 'shielding',
 'property',
 'is',
 'now',
 'growing',
 'rapidly',
 'mainly',
 'in',
 'the',
 'U.S.',
 'Plastic',
 'photochromic',
 'sunglasses',
 'which',
 'are',
 'capable',
 'of',
 'controlling',
 'glare',
 'shielding',
 'property',
 'due',
 'to',
 'variations',
 'in',
 'transmittance',
 'according',
 'to',
 'surrounding',
 'brightness',
 'and'

In [161]:
chunk_size = 100 # Chunk size in characters
gathered_tokenized = []
current_string_length = 0
current_words = []
i = 0
for word in description_words:
    if len(word) + current_string_length > chunk_size:
        to_tokenize = ' '.join(current_words)
        tokenized = clip.tokenize(to_tokenize)[0]
        nonzero_tokens = tokenized[tokenized.nonzero()[:,0]]
        gathered_tokenized.append(nonzero_tokens)
        current_words = [word]
        current_string_length = len(word)
    else:
        current_words.append(word)
        current_string_length += len(word)
all_tokenized = torch.cat(gathered_tokenized)


In [198]:
abstract_encoding = encode_text(abstract, model)
description_encoding = encode_text(description, model)
claims_encoding = encode_text(claims, model)

In [201]:
abstract_encoding/torch.linalg.norm(abstract_encoding) @ description_encoding/torch.linalg.norm(description_encoding)

tensor(0.8545, device='cuda:0', dtype=torch.float16)

In [252]:
foo = all_tokenized[:50]
len(pad(foo, (0,77-50)))

77

In [183]:
unfolded_text.shape

torch.Size([1231, 77])

In [122]:
len('1,2-butanediol, 1,3-butanediol, 1,4-butanediol, 1,5-pentanediol, 1,6-hexanediol, 1,8-octanediol, 1,9-nonanediol, 1,10-decanediol, 3-methyl-1,5-pentanediol, 2-ethyl-4-butyl-1,3-propanediol')

187

In [168]:
text_features = model.encode_text(all_tokenized[:77].unsqueeze(0).to(device=device))

In [185]:
with torch.no_grad():
    encodings = model.encode_text(unfolded_text.to(device=device))

In [189]:
encodings.sum(dim=0).shape

torch.Size([512])

In [64]:
tok_description = clip.tokenize(description)

RuntimeError: Input TECHNICAL FIELD
[0001] The present invention relates to a laminate which is used to produce lenses such as polarizing lenses having polarization properties and sunglass lenses having photochromic properties and an optical article comprising the laminate.
BACKGROUND ART
[0002] Demand for plastic substrates composed of polycarbonate having transparency and excellent impact resistance for use in sunglasses having glare shielding property is now growing rapidly mainly in the U.S. Plastic photochromic sunglasses which are capable of controlling glare shielding property due to variations in transmittance according to surrounding brightness and the amount of ultraviolet light by combining photochromic properties are becoming popular rapidly. These sunglasses are now manufactured by the following method.
[0003] For example, as means for forming a functional layer having glare shielding property between two plastic sheets or films, there is known a method in which a photochr

In [55]:
names[0].filename

'EP3399368.A1/patent_info.json'